In [2]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
from sentence_transformers import SentenceTransformer
dataset = load_dataset("ag_news")

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')

data_text = model.encode(dataset['train']['text'])

data_label = np.array(dataset['train']['label'])

In [4]:
val_text = data_text[108000:]
train_text = data_text[:108000]

val_label = data_label[108000:]
train_label = data_label[:108000]
print(train_text.shape)


(108000, 768)


In [5]:
from sequential import Sequential
from Dropout import DropLayer
from Dense import DenseLayer
model_mlp = Sequential([
    DenseLayer(768, 256, activation=tf.nn.relu),
    DropLayer(0.45),
    DenseLayer(256,512, activation=tf.nn.relu),
    DropLayer(0.45),
    DenseLayer(512, 4, initializer=tf.zeros)
])


ModuleNotFoundError: No module named 'dense'

In [ ]:
def get_loss(labels, logits):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)
    )
def get_accuracy(correct_label, est_output):
    model_mlp.isTrain = False
    est_output = tf.math.argmax(est_output, axis=1)
    num_correct = tf.equal(est_output, correct_label)
    accuracy = tf.reduce_mean(tf.cast(num_correct, tf.float32)) * 100.0
    model_mlp.isTrain = True
    return accuracy

from AdamW import AdamW

In [ ]:
optimizer_custom = AdamW(learning_rate = 0.01, weight_decay=0.02)
rng = tf.random.get_global_generator()
rng.reset_from_seed(0x43966E87BD57227011B5B03B58785EC1)
model_mlp.train_model(optimizer_custom, train_text, train_label, val_text, val_label, 1200, get_accuracy, get_loss, rng )


In [ ]:
import tensorflow as tf
model_keras = tf.keras.Sequential(layers=[
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.45, seed=3489024),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.45, seed=3489024),
    tf.keras.layers.Dense(4, activation='softmax')
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
loss = tf.keras.losses.SparseCategoricalCrossentropy()


model_keras.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

y_train = tf.keras.utils.to_categorical(dataset['train']['label'])


In [ ]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.01,weight_decay=0.02)
model_keras.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model_keras.fit(data_text, data_label, validation_split=0.10, epochs=10, batch_size=1024)


In [ ]:
y_test = np.array(dataset['test']['label'])
data_test = model.encode(dataset['test']['text'])



In [ ]:
model_keras.evaluate(data_test, y_test)
print("non-keras accuracy: ", get_accuracy(y_test, model_mlp(data_test)))